In [1]:
# import all packages needed
import string, math
import numpy as np
import pandas as pd
from matplotlib import pyplot
from base64 import b64decode as decode

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

## Data Processing / Cleaning

In [38]:
# use class base64 to decode waveform data
def to_array(wf):
    barr = bytearray(decode(wf))
    vals = np.array(barr)
    return vals.view(np.int16)

# read in data
exam_data = pd.read_csv("data/d_exam.csv").drop(columns = ["site_num", "patient_id_edit"])
waveform_data = pd.read_csv("data/d_waveform.csv")
lead_data = pd.read_csv("data/d_lead_data.csv").drop(columns = ["exam_id"])
diagnosis_data = pd.read_csv("data/d_diagnosis.csv").drop(columns = ["user_input"])

# add decoded data as a column to lead data
waveforms = list(lead_data['waveform_data'])
lead_data['decoded_waveform'] = [to_array(i) for i in waveforms]

# merge waveform data and lead data
waveform_lead = lead_data.merge(waveform_data, how = "left", left_on = "waveform_id", right_on = "waveform_id", suffixes = (None, None))

#  sort by exam id and lead id
waveform_lead.sort_values(by = ["waveform_id", "lead_id"], inplace = True)

waveform_lead.loc[:, ['exam_id', 'lead_id', 'decoded_waveform', 'waveform_type']]
waveform_lead

,lead_data_id,waveform_id,WavfmType,lead_id,lead_byte_count_total,lead_time_offset,waveform_data,lead_sample_count_total,lead_amplitude,lead_units,...,exam_id,waveform_type,number_of_leads,Waveform_Start_Time,Sample_Type,Sample_Base,Sample_Exponent,High_Pass_Filter,Low_Pass_Filter,AC_Filter
10,9078054,1095618,NaN,I,5000,0,+P/4//j/+P/4//j/+P/5//r/+//8//z//P/7//r/+f/4/...,2500,4.88,MICROVOLTS,...,549871,Rhythm,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE
15,9081703,1095618,NaN,II,5000,0,9v/2//b/8//w//D/8P/x//L/8//0//T/9P/z//L/8f/w/...,2500,4.88,MICROVOLTS,...,549871,Rhythm,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE
8,9074278,1095618,NaN,V1,5000,0,/v/+//7//v/+////AAAAAAAAAQACAAIAAgACAAIAAgACA...,2500,4.88,MICROVOLTS,...,549871,Rhythm,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE
1,9066887,1095618,NaN,V2,5000,0,9v/1//T/9P/0//T/9P/0//T/9f/2//b/9v/2//b/9v/2/...,2500,4.88,MICROVOLTS,...,549871,Rhythm,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE
18,9082771,1095618,NaN,V3,5000,0,7v/u/+7/7f/s/+z/7P/t/+7/7v/u/+7/7v/u/+7/7v/u/...,2500,4.88,MICROVOLTS,...,549871,Rhythm,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,9187141,1109067,NaN,V4,1200,0,KAApACoAKwAsACwALQAtAC4ALgAuAC4ALgAuAC4ALgAvA...,600,4.88,MICROVOLTS,...,554080,Rhythm,8,0,CONTINUOUS_SAMPLES,500,0,16,150,NONE
152,9190675,1109067,NaN,V5,1200,0,FgAXABkAGQAbABsAGwAbABsAGwAbABwAHQAeAB4AHgAfA...,600,4.88,MICROVOLTS,...,554080,Rhythm,8,0,CONTINUOUS_SAMPLES,500,0,16,150,NONE
155,9177603,1109067,NaN,V5,10000,0,+v/6//r/+v/7//z//f/+//z//P/8//z//v/+//7//v/+/...,5000,4.88,MICROVOLTS,...,554080,Rhythm,8,0,CONTINUOUS_SAMPLES,500,0,16,150,NONE
140,9172851,1109067,NaN,V6,10000,0,7v/u/+7/7v/x//L/8//0//T/9P/0//T/9P/0//T/9P/0/...,5000,4.88,MICROVOLTS,...,554080,Rhythm,8,0,CONTINUOUS_SAMPLES,500,0,16,150,NONE


In [48]:
# concatenate all leads into a single array
waveform_lead_concat = waveform_lead.groupby(["exam_id", "waveform_type"])['decoded_waveform'].apply(lambda x: tuple(x)).reset_index()
waveform_lead_concat
# remove irregular observations, concat tuple into numpy array
waveform_lead_concat = waveform_lead_concat.drop([12,17], axis = 0)
waveform_lead_concat['decoded_waveform'] = waveform_lead_concat['decoded_waveform'].apply(lambda x: np.vstack(x))

waveform_lead_rhythm = waveform_lead_concat[waveform_lead_concat['waveform_type'] == "Rhythm"]
#waveform_lead_median = waveform_lead_concat[waveform_lead_concat['waveform_type'] == "Median"]

waveform_lead_rhythm#["decoded_waveform"]

,exam_id,waveform_type,decoded_waveform
1,548759,Rhythm,"[[4, 3, 2, -1, -4, -4, -4, -4, -4, -7, -10, -8..."
3,549871,Rhythm,"[[-8, -8, -8, -8, -8, -8, -8, -7, -6, -5, -4, ..."
5,550602,Rhythm,"[[-22, -20, -18, -16, -14, -14, -14, -12, -10,..."
7,551485,Rhythm,"[[46, 45, 44, 42, 40, 35, 30, 26, 22, 18, 14, ..."
9,552077,Rhythm,"[[-7, -4, -1, -6, -10, -12, -14, -11, -11, -14..."
11,552856,Rhythm,"[[-32, -32, -32, -33, -34, -34, -34, -33, -32,..."
14,553115,Rhythm,"[[-8, -5, -2, -2, -2, -5, -8, -8, -8, -7, -6, ..."
16,553528,Rhythm,"[[-12, -12, -12, -12, -12, -12, -12, -12, -12,..."


In [47]:
# Adding the labels/sentences
exams = diagnosis_data["exam_id"].unique()

# Let's look over this tomorrow
diagnosis_data = diagnosis_data[diagnosis_data['Original_Diag'] == 1].dropna()
#searchfor = ['previous', 'unconfirmed', 'compared', 'interpretation', 'significant']
#diagnosis_data = diagnosis_data.loc[diagnosis_data['Full_text'].str.contains('|'.join(searchfor)) != 1]
#

diagnosis_data.sort_values(by=["exam_id", "statement_order"], inplace=True)
diagnoses = []
curr_id = 0
curr_string = ""
for i, row in diagnosis_data.iterrows():
    if row["statement_order"] == 1 and curr_string != "":
        curr_string = curr_string.lower().translate(str.maketrans('', '', string.punctuation))
        val = [curr_id, curr_string[1:]]
        diagnoses.append(val)
        curr_string = ""
        curr_id = row["exam_id"]

    if curr_id == 0:
        curr_id = row["exam_id"]
    
    curr_string += " " + row["Full_text"]

diagnosis_df = pd.DataFrame(diagnoses, columns = ['exam_id', 'diagnosis'])
waveform_lead_rhythm_diag = pd.merge(left=waveform_lead_rhythm, right=diagnosis_df, left_on='exam_id', right_on='exam_id')

#waveform_lead_rhythm_diag
waveform_lead_rhythm_diag

,exam_id,waveform_type,decoded_waveform,diagnosis
0,548759,Rhythm,"[[4, 3, 2, -1, -4, -4, -4, -4, -4, -7, -10, -8...",normal sinus rhythm low voltage qrs borderline...
1,549871,Rhythm,"[[-8, -8, -8, -8, -8, -8, -8, -7, -6, -5, -4, ...",sinus bradycardia otherwise normal ecg
2,550602,Rhythm,"[[-22, -20, -18, -16, -14, -14, -14, -12, -10,...",sinus tachycardia otherwise normal ecg
3,551485,Rhythm,"[[46, 45, 44, 42, 40, 35, 30, 26, 22, 18, 14, ...",normal sinus rhythm normal ecg
4,552077,Rhythm,"[[-7, -4, -1, -6, -10, -12, -14, -11, -11, -14...",normal sinus rhythm normal ecg
5,552856,Rhythm,"[[-32, -32, -32, -33, -34, -34, -34, -33, -32,...",normal sinus rhythm with sinus arrhythmia mini...
6,553115,Rhythm,"[[-8, -5, -2, -2, -2, -5, -8, -8, -8, -7, -6, ...",atrial fibrillation abnormal ecg normal sinus ...


In [5]:
unique_words = set()
for num, sentence in diagnoses:
    for word in sentence.split():
        unique_words.add(word)
print(unique_words)

{'for', 'be', 't', 'lvh', 'abnormality', 'otherwise', 'sinus', 'ischemia', 'criteria', 'atrial', 'borderline', 'may', 'with', 'ecg', 'normal', 'variant', 'consider', 'arrhythmia', 'rhythm', 'inferior', 'tachycardia', 'fibrillation', 'voltage', 'qrs', 'wave', 'bradycardia', 'minimal', 'low', 'abnormal'}


In [6]:
# split data into training and testing datasets
# y not included for now
def one_hot(x, dict_words):
    x = x.split(" ")
    array = []
    for i in x:
        array.append([1 if y == i else 0 for y in dict_words])
    return array

dict_words = list(unique_words)
#waveform_lead_rhythm_diag['diagnosis'] = waveform_lead_rhythm_diag['diagnosis'].apply(lambda x: one_hot(x, dict_words))

len(waveform_lead_rhythm_diag["diagnosis"][5])
#train_x, test_x, train_y, test_y = train_test_split(waveform_lead_rhythm_diag['decoded_waveform'], waveform_lead_rhythm_diag['diagnosis'], test_size = 0.1, random_state = 2021)
#train_x = torch.tensor(list(train_x)).float()
#train_x.shape

111

## Model 1 - Conv1D Encoder w/ LSTM Decoder

In [182]:
# HYPERPARAMETERS
J = 10 # max number of filters per class
LR = 1e-3

# define global max pooling
class global_max_pooling_1d(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        x, _ = torch.max(x, dim = 2)
        return(x)

# 1D grouped encoder model
encoder_conv = nn.Sequential()
encoder_conv.add_module('initial_norm', nn.BatchNorm1d(8))
encoder_conv.add_module('conv_1', nn.Conv1d(in_channels = 8, out_channels = 8, groups = 8, kernel_size = 5, padding = 2))
for i in range(2, (J+2), 2):
    if (i-2) == 0: 
        prev = 8
    else:
        prev = (i-2)*8
    encoder_conv.add_module('conv_{num}'.format(num = int(i / 2 + 1)), nn.Conv1d(in_channels = prev, out_channels = i*8, groups = 8, kernel_size = 5, padding = 2, stride = 3))
    encoder_conv.add_module('activation_{num}'.format(num = int(i / 2 + 1)), nn.ELU())
    encoder_conv.add_module('batch_norm_{num}'.format(num = int(i / 2 + 1)), nn.BatchNorm1d(i*8))
    
encoder_conv.add_module('final_conv', nn.Conv1d(in_channels = J * 8, out_channels = 8, groups = 8, kernel_size = 5, padding = 2))
encoder_conv.add_module('max_pool', nn.MaxPool1d(kernel_size = 5, padding = 2, stride = 1))
encoder_conv.add_module('reshape', nn.MaxPool1d(kernel_size = 5, padding = 2, stride = 1))


# summarize model, verify output is of desired shape
print(encoder_conv)
print(encoder_conv(train_x).shape)

Sequential(
  (initial_norm): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_1): Conv1d(8, 8, kernel_size=(5,), stride=(1,), padding=(2,), groups=8)
  (conv_2): Conv1d(8, 16, kernel_size=(5,), stride=(3,), padding=(2,), groups=8)
  (activation_2): ELU(alpha=1.0)
  (batch_norm_2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_3): Conv1d(16, 32, kernel_size=(5,), stride=(3,), padding=(2,), groups=8)
  (activation_3): ELU(alpha=1.0)
  (batch_norm_3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_4): Conv1d(32, 48, kernel_size=(5,), stride=(3,), padding=(2,), groups=8)
  (activation_4): ELU(alpha=1.0)
  (batch_norm_4): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_5): Conv1d(48, 64, kernel_size=(5,), stride=(3,), padding=(2,), groups=8)
  (activation_5): ELU(alpha=1.0)
  (batch_norm_5): BatchNorm1d(64, eps=1e-05, momentum=0.1,

## Model 2 - LSTM Encoder w/ Huggingface Decoder

In [30]:
# define hyperparameters 
hidden_layers = 512
embedding_dim = 8
num_words = len(unique_words)

class ECG_LSTM(nn.Module):
    def __init__(self, encoder, h_dim, e_dim, word_list_length):
        super(ECG_LSTM, self).__init__()
        self.encoder = encoder
        self.lstm = nn.LSTM(e_dim, h_dim)
        self.linear = nn.Linear(h_dim, word_list_length)
        
    def forward(self, seq):
        seq_embedded = self.encoder(seq)
        final_hidd, _ = self.lstm(seq_embedded)
        dec_seq = self.linear(final_hidd)
        return F.log_softmax(dec_seq)
    
lstm_dec = ECG_LSTM(encoder_conv, hidden_layers, embedding_dim, num_words)
lstm_dec(train_x).shape

C:\Users\alber\anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


torch.Size([7, 8, 500])

## Model 3 - Basic Transformer Architecture with Multi-Head Attention

# Transformer testing

In [79]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer

class ECGTransformerEncoder(nn.Module):
    # Takes the ECG discrete signals sequence and maps into a probability distribution of diagnosis
    # For working/verification purposes
    def __init__(self, vector_size, n_inputs, n_heads, hidden_linear_dim, n_layers, dropout):
        super(ECGTransformerEncoder, self).__init__()
        self.model_type = "Transformer"
        self.positional_encoder = PositionalEncoder(n_inputs, dropout)
        
        #Since our data is already discrete numbers, might need some tweaking for this
        self.embedder = SignalEmbedder(n_inputs, vector_size)
                                      #39        64
        
        
        self.encoder = TransformerEncoder(
            TransformerEncoderLayer(n_inputs, n_heads, hidden_linear_dim, dropout),
            n_layers)
        
        self.n_inputs = n_inputs
        self.n_layers = n_layers
        
        # Simple linear decoder
        #self.decoder = nn.Linear(n_inputs, 20)
        self.init_weights()
        
    def init_weights(self):
        #self.embedder.weight.data.uniform_(-.1, .1)
        #self.decoder.bias.data.zero_()
        #self.decoder.weight.data.uniform_(-.1, .1)
        pass
        
    def forward(self, x):
        x = self.embedder(x) # * math.sqrt(self.n_inputs)
        x = self.positional_encoder(x)
        x = self.encoder(x)
        #x = self.decoder(x)
        return x


class SignalEmbedder(nn.Module):
    # Necessary to convert the signal into "word" vectors for transformer processing.
    # Currently a simple group and slice method, but will modify later for multi-channel inputs
    
    def __init__(self, num_slices, size_of_slice):
        super(SignalEmbedder, self).__init__()
        self.num_slices = num_slices
        self.size_of_slice = size_of_slice
        
    def forward(self, x):
        x = x[: self.num_slices * self.size_of_slice]
        x = x.reshape((self.num_slices, self.size_of_slice))
        return x

class PositionalEncoder(nn.Module):
    # Necessary to store positional data about the input data
    def __init__(self, d_model, dropout=0.1, max_len=1):
        super(PositionalEncoder, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pos_encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        
        divisor = torch.exp(torch.arange(0, d_model, 2).float() * (- math.log(10000.0) / d_model))
        
        pos_encoding[:, 0::2] = torch.sin(position * divisor)
        pos_encoding[:, 1::2] = torch.cos(position * divisor)
        pos_encoding = pos_encoding.unsqueeze(0).transpose(0, 1)
        
        self.register_buffer("pos_encoding", pos_encoding)

        
    def forward(self, x):
        x = x + self.pos_encoding[:x.size(0), :]
        return self.dropout(x)

In [95]:
# Training pipeline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model to set to
model = ECGTransformerEncoder(vector_size=50, n_inputs=50, n_heads=10, hidden_linear_dim=2048, n_layers=4, dropout=0.3).to(device)

# Training params
loss_function = nn.CrossEntropyLoss()
lr = .001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma = 0.95)

data = []
for arr in waveform_lead_rhythm_diag["decoded_waveform"]:
    #print(arr)
    data.append(arr[0])

data = torch.from_numpy(np.array(data)).type(torch.LongTensor)
print(data.shape)
for i in data:
    value = model(i).squeeze(0)
    print(value)
    print(value.shape)


torch.Size([7, 2500])
tensor([[ 1.2541,  0.6125, -0.5987,  ...,  2.0761, -0.4451, -0.8662],
        [ 0.4469, -0.4472, -0.7344,  ...,  0.0152, -1.3190, -1.1759],
        [ 0.2178,  0.4010, -0.3895,  ..., -0.3051, -0.1499,  1.4969],
        ...,
        [ 1.0541, -0.0703, -0.1071,  ..., -1.8216, -0.0028,  0.6536],
        [-0.5618, -0.3496, -0.6838,  ..., -1.4583, -1.6458,  0.9673],
        [ 0.7857, -0.8160, -0.3676,  ...,  1.5233, -2.0489, -2.3213]],
       grad_fn=<SqueezeBackward1>)
torch.Size([50, 50])
tensor([[-0.2458, -0.3301, -1.9814,  ..., -0.6758,  0.6204, -0.9355],
        [-0.6897,  1.3317, -0.0984,  ...,  0.4932, -0.2818, -0.4940],
        [-0.2679, -1.1653,  0.3712,  ..., -0.9666,  0.2672,  0.8188],
        ...,
        [ 0.2182, -0.9000,  1.1068,  ..., -0.4687, -0.9484,  0.3853],
        [-0.9080, -1.1475, -0.4045,  ..., -0.0725, -1.3122, -1.0173],
        [-1.5262, -0.2151, -0.7750,  ..., -1.0196, -1.0730, -0.7755]],
       grad_fn=<SqueezeBackward1>)
torch.Size([50, 50]

## Model 4 - FNET Transformer Architecture

## Model 5 - FNET/Basic Mixup Architecture 